<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 

<a href="http://wikistat.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/wikistat.jpg" style="max-width: 250px; display: inline"  alt="Wikistat"/></a>

<a href="http://www.math.univ-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo_imt.jpg" style="float:right; max-width: 250px; display: inline" alt="IMT"/> </a>
</center>

# Modèles linéaires en grande dimension avec <a href="https://cran.r-project.org/"><img src="https://cran.r-project.org/Rlogo.svg" style="max-width: 40px; display: inline" alt="R"/></a>

** Objectifs **
Comparaison sur le même jeu de données des qualités de prévision de plusieurs modèles obtenus par:
- Modèle linéaire
-  Algorithmes de sélection de modèles (critères AIC, BIC)
- Régularisation (ridge, Lasso, Elastic Net)
- Régression sur composantes (PCR, PLS)


## Les données

Nous allons utiliser les données `Prostate` du package **`lasso 2 `** de R.

 These data come from a study that examined the correlation between the
  level of prostate specific antigen and a number of clinical measures
  in men who were about to receive a radical prostatectomy.  It is data
  frame with 97 rows and 9 columns.

** Source : Stamey, T.A., Kabalin, J.N., McNeal, J.E., Johnstone, I.M., Freiha,
  F., Redwine, E.A. and Yang, N. (1989). *Prostate specific antigen in the diagnosis and treatment of
  adenocarcinoma of the prostate: II. radical prostatectomy treated
  patients* ,  Journal of Urology, {141}(5), 1076--1083. **

Liste des 9 variables la dernière est à expliquer : 
- **`lcavol`** :  log(cancer volum)
- **`lweight`** :  log(prostate weight)
- **`age`** :   age
- **`lbph `** :  log(benign prostatic hyperplasia amount)
- **`svi`** :  seminal vesicle invasion
- **`lcp`** :  log(capsular penetration)
- **`lweight`** :  log(prostate weight)
- **`gleason`** : Gleason score
- **`pgg45 `** :   percentage Gleason scores 4 or 5

- **`lpsa `** :  log(prostate specific antigen)


Les variables **`svi`**  et **`gleason`** sont qualitatives.


### Lecture des données

In [ ]:
library(lasso2)
data(Prostate)
# Une seule valeur 8 du score de Gleason ; on regroupe  les modalités 8 et 9 ensembles
Prostate$gleason[Prostate$gleason==8]=9
# On transforme svi et gleason en variables qualitatives
Prostate$svi=as.factor(Prostate$svi)
Prostate$gleason=as.factor(Prostate$gleason)
summary(Prostate)

### Description des données

In [ ]:
dim(Prostate)
names(Prostate)
summary(Prostate)
cor(Prostate[,-c(5,7)])
hist(Prostate$lpsa,10)


### Création d'un échantillon d'apprentissage et d'un échantillon test

In [ ]:
#Les valeurs de lpsa sont rangées par ordre croissant
#On conserve 1/4 des données pour l'échantillon test 
ind.test=4*c(1:22)
Prostate.app=Prostate[-ind.test,]
Prostate.test=Prostate[c(ind.test),]
dim(Prostate.test)
dim(Prostate.app)
ntest=length(Prostate.test$lpsa)
napp=length(Prostate.app$lpsa)
summary(Prostate.app)
summary(Prostate.test)

## Modèle linéaire complet

### Estimation du modèle et graphes des résidus

In [ ]:
# Une fonction utile de graphe des résidus.

plot.res=function(x,y,titre="")
{
plot(x,y,col="blue",ylab="Résidus",
   xlab="Valeurs predites",main=titre)
abline(h=0,col="green")
}

In [ ]:
modlin=lm(lpsa~., data=Prostate.app)
summary(modlin) # noter les p-valeurs

#Residus 
res=residuals(modlin)

#Regroupement des graphiques sur la meme page
par(mfrow=c(1,2))
hist(residuals(modlin))
qqnorm(res)
qqline(res, col = 2)
 
par(mfrow=c(1,1)) # retour au graphique standard
plot.res(predict(modlin),res)

### Erreur d'apprentissage et erreur de généralisation

In [ ]:
#Erreur d'apprentissage

mean(res**2)

#Erreur sur l'échantillon test

pred.test=predict(modlin, newdata=Prostate.test)
res.test=pred.test-Prostate.test$lpsa
mean(res.test**2)

### Nouvelle paramétrisation 

Afin de faciliter l'interprétation des résultats concernant les variables qualitatives, on introduit une nouvelle paramétrisation à l'aide de contrastes. 

Par défaut, la référence est prise pour la valeur 0 de  **`svi`** et 6 de  **`gleason`**, qui sont les plus petites valeurs. Les paramètres indiqués pour les variables svi1, gleason 7, 8 et 9 indiquent l'écart estimé par rapport à cette référence. Il est plus intéressant en pratique de se référer à la moyenne des observations sur toutes les modalités des variables qualitatives, et d'interpréter les coefficients comme des écarts à cette moyenne. 



In [ ]:
contrasts(Prostate.app$svi)=
contr.sum(levels(Prostate.app$svi)) 
 
contrasts(Prostate.app$gleason)=
contr.sum(levels(Prostate.app$gleason)) 

modlin2=lm(lpsa~., Prostate.app)
summary(modlin2)
 
# Attention au nom des variables : gleason1 =6,  gleason2 =7, gleason3 =8-9 (pas affiché),
#la somme des coefficients associés à ces variables est nulle
#svi1=0, svi2=1 (pas affiché), la somme des deux coefficients est nulle. 


## Sélection de modèle par sélection de variables
### Sélection par AIC et backward

In [ ]:
library(MASS)

modselect_b=stepAIC(modlin2,~.,trace=TRUE,
direction=c("backward"))

summary(modselect_b)  

### Sélection par AIC et forward

In [ ]:
mod0=lm(lpsa~1,data=Prostate.app)

modselect_f=stepAIC(mod0,lpsa~lcavol+lweight
+age+lbph+svi+lcp+gleason+pgg45,
data=Prostate.app,trace=TRUE,direction=c("forward"))

summary(modselect_f)


### Sélection par AIC et stepwise

In [ ]:

modselect=stepAIC(modlin2,~.,trace=TRUE,
direction=c("both"))
#both est l'option par défaut

summary(modselect)


### Sélection par BIC et stepwise

In [ ]:
#  k=log(napp) pour BIC au lieu de AIC.

modselect_BIC=stepAIC(modlin2,~.,trace=TRUE,
direction=c("both"),k=log(napp))

summary(modselect_BIC)
#Le modèle sélectionné est plus parcimonieux


### Calcul de l'erreur d'apprentissage 

In [ ]:
#Modèle stepwise AIC

mean((predict(modselect)-Prostate.app[,"lpsa"])**2)

#Modèle stepwise BIC
mean((predict(modselect_BIC)-Prostate.app[,"lpsa"])**2)


** Q.** Commenter ces résultats : laquelle des 2 erreurs est la plus grande.  Pouvait on le prévoir ?  

### Calcul de l'erreur de test

In [ ]:
#Modèle stepwise AIC

mean((predict(modselect,newdata=Prostate.test)-Prostate.test[,"lpsa"])**2)

#Modèle stepwise BIC

mean((predict(modselect_BIC,newdata=Prostate.test)-Prostate.test[,"lpsa"])**2)

** Q.** Commenter ces résultats : laquelle des 2 erreurs est la plus grande.  Pouvait on le prévoir ? 

## Sélection de modèle par pénalisation Ridge

### Comportement des coefficients

Calcul des coefficients pour différentes valeurs du paramètre lambda

In [ ]:
library(MASS)
mod.ridge=lm.ridge(lpsa~.,data=Prostate.app,
lambda=seq(0,20,0.1))
par(mfrow=c(1,1))
plot(mod.ridge)

matplot(t(mod.ridge$coef),lty=1:3,type='l',col=1:10)
legend("top",legend=rownames(mod.ridge$coef),
col=1:10,lty=1:3)

### Pénalisation optimale par validation croisée

In [ ]:
select(mod.ridge) # noter la valeur puis estimer

In [ ]:
mod.ridgeopt=lm.ridge(lpsa ~  .,data=Prostate.app,lambda=9.8)


** Q. ** Automatiser le calcul du modèle avec la valeur optimale de lambda  

### Prévision  sur l'échantillon d'apprentissage

La  fonction `predict.ridgelm` n'existe pas, il faut calculer les valeurs prédites à partir des coefficients.

In [ ]:
#Coefficients du modèle sélectionné :

coeff=coef(mod.ridgeopt)

#On crée des vecteurs pour les variables  qualitatives

svi0.app=1*c(Prostate.app$svi==0)
svi1.app=1-svi0.app
gl6.app=1*c(Prostate.app$gleason==6)
gl7.app=1*c(Prostate.app$gleason==7)
gl9.app=1*c(Prostate.app$gleason==9)

#variables quantitatives

lcavol.app=Prostate.app$lcavol
lweight.app=Prostate.app$lweight
age.app=Prostate.app$age
lbph.app=Prostate.app$lbph
lcp.app=Prostate.app$lcp
pgg45.app=Prostate.app$pgg45


#Calcul des valeurs prédites
fit.rid=rep(coeff[1],napp)+coeff[2]*lcavol.app+coeff[3]*lweight.app+coeff[4]*age.app+coeff[5]*lbph.app+coeff[6]*svi0.app-coeff[6]*svi1.app+coeff[7]*lcp.app+coeff[8]*gl6.app+coeff[9]*gl7.app-(coeff[8]+coeff[9])*gl9.app+coeff[10]*pgg45.app


** Tracé des valeurs prédites en fonctions  des valeurs observées : **

In [ ]:
plot(Prostate.app$lpsa,fit.rid)
abline(0,1)

In [ ]:
# Calcul et tracé des résidus
res.rid=fit.rid-Prostate.app[,"lpsa"]
plot.res(fit.rid,res.rid,titre="")

In [ ]:
#Erreur d'apprentissage
mean(res.rid**2)

### Prévision sur l'échantillon test

In [ ]:
svi0.test=1*c(Prostate.test$svi==0)
svi1.test=1-svi0.test
gl6.test=1*c(Prostate.test$gleason==6)
gl7.test=1*c(Prostate.test$gleason==7)
gl9.test=1*c(Prostate.test$gleason==9)

ntest=length(Prostate.test$lpsa)
prediction=rep(coeff[1],ntest)+coeff[2]*Prostate.test$lcavol+coeff[3]*Prostate.test$lweight+coeff[4]*Prostate.test$age+coeff[5]*Prostate.test$lbph+coeff[6]*svi0.test-coeff[6]*svi1.test+coeff[7]*Prostate.test$lcp+coeff[8]*gl6.test+coeff[9]*gl7.test-(coeff[8]+coeff[9])*gl9.test+coeff[10]*Prostate.test$pgg45


** Erreur sur l'échantillon test **

In [ ]:
mean((Prostate.test[,"lpsa"]-prediction)^2)


In [ ]:
par(mfrow=c(1,1))
plot(mod.ridge)
abline(v=9.8)

** Q. ** Commenter les résultats et comparer avec les modèles précédents

## Sélection de modèle par pénalisation Lasso avec la  Librairie Lasso2 

### Construction du modèle

In [ ]:
library(lasso2)
l1c.P <- l1ce(lpsa ~ ., Prostate.app, bound=(1:100)/100, absolute.t=FALSE)

La borne est ici relative, elle correspond à une certaine proportion de la norme $\mathbb{L}_1$ du vecteur des 
 coefficients des moindres carrés. Une borne égale à $1$ correspond donc  à l'absence de pénalité, on retrouve 
l'estimateur des  moindres carrés. 

### Visualisation des coefficients : chemins de régularisation

In [ ]:
coefficients=coef(l1c.P)
plot(l1c.P,col=1:11,lty=1:3,type="l")
legend("topleft",legend=colnames(coefficients),col=1:11,lty=1:3)

#On supprime le terme constant
penalite_relative=c(1:100)/100
matplot(penalite_relative,coefficients[,-1],lty=1:3,type='l',col=1:10)
legend("topleft",legend=colnames(coefficients[,-1]),col=1:10,lty=1:3)

### Sélection de la pénalité par validation croisée

In [ ]:

vc=gcv(l1c.P)
crit.vc=vc[,"gcv"]
bound_opt=vc[which.min(crit.vc),"rel.bound"]

l1c.opt <- l1ce(lpsa ~ ., Prostate.app, 
bound=bound_opt, absolute.t=FALSE)
coef=coef(l1c.opt)

In [ ]:
### Erreur d'apprentissage et erreur de généralisation

In [ ]:
#erreur apprentissage
fit=fitted(l1c.opt)
mean((fit-Prostate.app[,"lpsa"])^2)


#Erreur sur l'échantillon test}
prediction=predict(l1c.opt,newdata=Prostate.test)
mean((prediction-Prostate.test[,"lpsa"])^2)

## Sélection de modèle par pénalisation Lasso avec la  Librairie glmnet

L'utilisation de la librairie **`glmnet`** fournit des résultats plus rapides, ce qui peut s'avérer
important pour des données de grande dimension. Par contre, on ne peut pas traiter à priori des variables qualitatives.
 Nous allons donc devoir créer des vecteurs avec des variables indicatrices des diverses modalités 
pour les variables qualitatives. Nous ne prendrons pas en compte les contrastes. 



### Mise en forme des variables

In [ ]:
#on construit une matrice xx.app d'apprentissage et xx.test de test
# On recharge les données qui sont quantitatives
data(Prostate)
Prostate$gleason[Prostate$gleason==8]=9
Prostate.app=Prostate[-ind.test,]
Prostate.test=Prostate[c(ind.test),]

x.app=Prostate.app[,-9]
y.app=Prostate.app[,9]
x.app=as.matrix(x.app)

#on construit une matrice avec les vecteurs indicatrices 
gl7.app=1*c(Prostate.app$gleason==7)
gl9.app=1*c(Prostate.app$gleason==9)

xx.app=matrix(0,ncol=9,nrow=nrow(x.app))
xx.app[,1:6]=as.matrix(x.app[,1:6])
xx.app[,7:8]=cbind(gl7.app,gl9.app)
xx.app[,9]=cbind(x.app[,8])

#on nomme les colonnes avec le noms des variables
colnames(xx.app)=c("lcavol","lweight", "age","lbph","svi1","lcp","gl7","gl9","pgg45")

#on fait de meme pour l'echantillon test
x.test=Prostate.test[,-9]
y.test=Prostate.test[,9]
x.test=as.matrix(x.test)
gl7.test=1*c(Prostate.test$gleason==7)
gl9.test=1*c(Prostate.test$gleason==9)

#on construit une matrice avec les vecteurs indicatrices 
xx.test=matrix(0,ncol=9,nrow=nrow(x.test))
xx.test[,1:6]=x.test[,1:6]
xx.test[,7:8]=cbind(gl7.test,gl9.test)
xx.test[,9]=x.test[,8]
colnames(xx.test)=colnames(xx.app)


### Construction du modèle

In [ ]:
library(glmnet)
out.lasso <- glmnet(xx.app,y.app)
l=length(out.lasso$lambda)
b=coef(out.lasso)[-1,1:l]

### Visualisation des coefficients  : chemins de régularisation

In [ ]:

matplot(t(as.matrix(out.lasso$beta)),type='l',
col=1:10,lty=1:3)
legend("topleft",legend=colnames(xx.app),
col=1:10,lty=1:3)
title("Lasso")

### Sélection de la pénalité par validation croisée

In [ ]:
a=cv.glmnet(xx.app,y.app)

lambda.opt=a$lambda.min
app=glmnet(xx.app,y.app,lambda=lambda.opt)

In [ ]:
### Erreur d'apprentissage

In [ ]:
appr=predict(app,newx=xx.app)
mean((appr-Prostate.app[,9])^2)

In [ ]:
### Erreur sur l'échantillon test

In [ ]:
pred=predict(app,newx=xx.test)
mean((pred-Prostate.test[,9])^2)

** Q. ** Comparer les résultats  et le modèle obtenu avec ce qui a été obtenu par la librairie Lasso 2. 

## Sélection de modèle par pénalisation avec Elastic Net

In [ ]:
# on peut faire varier avec le paramètre alpha de glmnet
out.elnet <- glmnet(xx.app,y.app,alpha=0.5)

a.elnet=cv.glmnet(xx.app,y.app,alpha=0.5)
lambda.opt=a.elnet$lambda.min
app=glmnet(xx.app,y.app,lambda=lambda.opt)

#erreur apprentissage
app.elnet=predict(a.elnet,newx=xx.app)
mean((app.elnet-Prostate.app[,9])^2)

#erreur de prédiction
predi.elnet=predict(a.elnet,newx=xx.test)
mean((predi.elnet-Prostate.test[,9])^2)


## Sélection de modèle projection sur composantes orthogonales

### Régression PLS

In [ ]:
data(Prostate)
Prostate$gleason[Prostate$gleason==8]=9
Prostate$svi=as.factor(Prostate$svi)
Prostate$gleason=as.factor(Prostate$gleason)
Prostate.app=Prostate[-ind.test,]
Prostate.test=Prostate[c(ind.test),]

library(pls) 
#nombre optimal de composantes par validation croisée
simpls= mvr(lpsa~.,data=Prostate.app, ncomp=9, validation="CV", method="simpls")
summary(simpls)
#graphique
plot(simpls)
abline(0,1)

#noter le nombre optimal de composantes 
plot(simpls,"val")

#Avec leave-one -out
simplsloo= mvr(lpsa~.,data=Prostate.app, ncomp=9, 
   validation="LOO", method="simpls")
summary(simplsloo)

#On sélectionne 6 composantes


In [ ]:
#Calcul des prévisions
predapp.pcr=predict(simpls,ncomp=5)
resapp.pcr=predapp.pcr-Prostate.app$lpsa

#Erreur d'apprentissage
mean(resapp.pcr**2)

#Valeurs prédites sur l'échantillon test
pred.pls=predict(simpls,newdata=Prostate.test,ncomp=5)
#Erreur de test
mean((pred.pls-Prostate.test[,"lpsa"])**2) 

### Régression sur composantes principales

In [ ]:

mod.pcr = pcr(lpsa~.,data=Prostate.app, ncomp=9, 
validation="CV")
summary(mod.pcr) 
# noter le nombre optimal
plot(mod.pcr,"val")
#On sélectionne 8 composantes



In [ ]:
#Calcul des prévisions
predapp.pls=predict(mod.pcr,ncomp=8)
resapp.pls=predapp.pls-Prostate.app$lpsa
#Erreur d'apprentissage
mean(resapp.pls**2)


#Valeurs prédites sur l'échantillon test
pred.pcr=predict(mod.pcr,newdata=Prostate.test,ncomp=8)
#Erreur de test
mean((pred.pcr-Prostate.test[,"lpsa"])**2) 